<a href="https://colab.research.google.com/github/So-yeon9/Tobigs20/blob/main/Week7/Week7_NLPBasic_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Basic Assignment
## 과제 : spam.csv를 활용하여 유의미한 해석을 도출해주세요!

In [73]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

## Load Data
- 보시면 아시다시피 spam.csv는 라벨이 있는 데이터입니다.
- 7주차 주제가 텍스트 기초인만큼 텍스트만 활용하셔도 되고 라벨까지 활용하셔서 모델을 돌려보셔도 좋습니다.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
spam = pd.read_csv('/content/drive/MyDrive/Tobigs20_week7_NLPBasic/spam.csv')

In [8]:
spam.iloc[5]['v2']

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [9]:
spam

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [80]:
spam.isna().sum().sum()

0

## Tokenizing


In [13]:
import nltk

In [16]:
# 예시 코드 코드
from nltk.tokenize import word_tokenize

nltk.download('punkt')
print(word_tokenize(spam.iloc[5]['v2']))

['FreeMsg', 'Hey', 'there', 'darling', 'it', "'s", 'been', '3', 'week', "'s", 'now', 'and', 'no', 'word', 'back', '!', 'I', "'d", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'Tb', 'ok', '!', 'XxX', 'std', 'chgs', 'to', 'send', ',', 'å£1.50', 'to', 'rcv']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
from nltk.tokenize import WordPunctTokenizer

print(WordPunctTokenizer().tokenize(spam.iloc[5]['v2']))

['FreeMsg', 'Hey', 'there', 'darling', 'it', "'", 's', 'been', '3', 'week', "'", 's', 'now', 'and', 'no', 'word', 'back', '!', 'I', "'", 'd', 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', '?', 'Tb', 'ok', '!', 'XxX', 'std', 'chgs', 'to', 'send', ',', 'å', '£', '1', '.', '50', 'to', 'rcv']


- WordPunctTokenizer 는 구두점 또한 별도로 분리함

In [19]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
print(text_to_word_sequence(spam.iloc[5]['v2']))

['freemsg', 'hey', 'there', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back', "i'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still', 'tb', 'ok', 'xxx', 'std', 'chgs', 'to', 'send', 'å£1', '50', 'to', 'rcv']


- it's, i'd 등을 하나의 토큰으로 분리하고 모든 문자를 자동으로 소문자로 바꿔줌으로써 의미 단위를 기준으로 토큰을 가장 잘 분리했다고 생각하여 text_to_word_sequence를 사용하기로 함.

## Embedding

- 수업에서 다룬 임베딩 방법에는 One-hot encoding, CBOW, Skip-gram 등이 있었습니다. 다양한 시도와 '비교' 결과를 함께 적어주세요! 파라미터를 조정해가는 과정도 해석에 도움이 될 수 있겠죠 :)

**1) One-hot encoding**

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts([spam.iloc[5]['v2']])
word_to_index = tokenizer.word_index
print(word_to_index)

encoded = tokenizer.texts_to_sequences([spam.iloc[5]['v2']])[0]
print(encoded)

{'to': 1, 'freemsg': 2, 'hey': 3, 'there': 4, 'darling': 5, "it's": 6, 'been': 7, '3': 8, "week's": 9, 'now': 10, 'and': 11, 'no': 12, 'word': 13, 'back': 14, "i'd": 15, 'like': 16, 'some': 17, 'fun': 18, 'you': 19, 'up': 20, 'for': 21, 'it': 22, 'still': 23, 'tb': 24, 'ok': 25, 'xxx': 26, 'std': 27, 'chgs': 28, 'send': 29, 'å£1': 30, '50': 31, 'rcv': 32}
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 1, 29, 30, 31, 1, 32]


In [45]:
from tensorflow.keras.utils import to_categorical

one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [75]:
np.array(one_hot).shape

(33, 33)

**2) Word2Vec**

CBOW

In [44]:
tokenized = [text_to_word_sequence(text) for text in spam['v2']]
tokenized[:2]

[['go',
  'until',
  'jurong',
  'point',
  'crazy',
  'available',
  'only',
  'in',
  'bugis',
  'n',
  'great',
  'world',
  'la',
  'e',
  'buffet',
  'cine',
  'there',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'u', 'oni']]

In [64]:
from gensim.models.word2vec import Word2Vec

cbow = Word2Vec(sentences=tokenized, window=3, min_count=3, workers=4, sg=0)
similarity = cbow.wv.most_similar("nice")
print(similarity)

[('night', 0.9992403984069824), ('had', 0.9991174936294556), ('very', 0.9990977644920349), ('dear', 0.9990636706352234), ('sleep', 0.9989791512489319), ('amp', 0.9989688992500305), ('today', 0.9989652633666992), ('people', 0.9989534616470337), ('nice', 0.9988996982574463), ('good', 0.9988722801208496)]


In [77]:
cbow.wv[0].shape

(100,)

In [68]:
cbow2 = Word2Vec(sentences=tokenized, window=10, min_count=3, workers=4, sg=0)
similarity2 = cbow2.wv.most_similar("nice")
print(similarity2)

[('dear', 0.9992220997810364), ('better', 0.9992168545722961), ('too', 0.9991483092308044), ('wish', 0.9991112947463989), ('always', 0.9991058111190796), ('being', 0.999089241027832), ('things', 0.9990818500518799), ('as', 0.9990376830101013), ('keep', 0.9990352392196655), ('some', 0.999034583568573)]


- window 크기를 3에서 10으로 늘리고 min_count는 유지하여 더 많은 주변 단어를 보고 학습하게 하였더니 nice와 유사한 단어에서 great, well, good 등이 제외되었지만 better 등의 단어가 추가되었음. 이것으로 보아 더 많은 주변 단어를 본다는 것은 학습량은 늘어나지만 부정확한 정보를 학습할 가능성도 그만큼 증가한다고 생각됨.

Skip-gram

In [69]:
sg = Word2Vec(sentences=tokenized, window=3, min_count=3, workers=4, sg=1)
similarity3 = sg.wv.most_similar("nice")
print(similarity3)

[('dear', 0.9831520318984985), ('as', 0.9801291227340698), ('having', 0.9792755246162415), ('amp', 0.9740644693374634), ('great', 0.9736491441726685), ('goes', 0.9713864922523499), ('sweet', 0.9704941511154175), ('wish', 0.9681700468063354), ('hope', 0.9673094749450684), ('lot', 0.9667811989784241)]


In [70]:
sg2 = Word2Vec(sentences=tokenized, window=10, min_count=3, workers=4, sg=1)
similarity4 = sg2.wv.most_similar("nice")
print(similarity4)

[('kids', 0.9412683844566345), ('many', 0.9386081695556641), ('colleagues', 0.9351361989974976), ('sad', 0.931580126285553), ('dreams', 0.9313327670097351), ('sweet', 0.9262563586235046), ('feeling', 0.9242072105407715), ('words', 0.9239532947540283), ('hello', 0.9228038191795349), ('full', 0.9210906028747559)]


- CBOW를 이용했을 때와 비교했을 때 같은 단어에 대해 유사하다고 나오는 단어의 구성이 상당히 다름.

## 본인이 도출해낸 해석을 적어주세요!

- 유사도, Wordcloud, 이진 분류 모델, Plot 뭐든 상관없으니 분명하고 인상적인 해석을 적어주시면 됩니다.